In [1]:
import numpy as np
# import xgboost as xgb
import pandas as pd
from datetime import datetime
from time import strptime
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
from numpy import loadtxt
# from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier

### define functions to clean data, extract features

In [2]:
dtrain=pd.read_csv('./train.csv')
dtest=pd.read_csv('./test.csv') #for final prediction
drating=pd.read_csv('./ratings.csv')
drmk=pd.read_csv('./remarks.csv')
drmk_supp=pd.read_csv('./remarks_supp_opp.csv')

/Users/rudransh/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# print(dtrain.head())
# print('')
# print(dtest.head())
# print('')
# print(drating.head())
# print('')
# print(drmk.head())
# print('')
# print(drmk_supp.head())

In [4]:
# print(len(dtrain))
# print(len(dtest))
# print(len(drating))
# print(len(drmk))
# print(len(drmk_supp))

In [5]:
# print((np.sum(dtrain['emp']>0)))
# print((np.sum(dtest['emp']>0)))
# print((np.sum(drating['emp']>0)))
# print((np.sum(drmk['emp']>0))) #has fake values
# print((np.sum(drmk_supp['emp']>0))) #has fake values

In [6]:
def basicClean(df): # removes entries with emp<0 and generates a unique Id comp+emp 
    df=df[df['emp']>0].reset_index()
    df['compemp']=df['comp']+df['emp'].apply(str)
    return df

def sep_train(df): #to separate features and labels
    Y=df[['compemp','left']]
    X=df[['id','emp','comp','lastratingdate']]
    return X,Y
# thought of additional features, like how many people leave particular company


In [7]:
def clean_dt(df): #same for test and train
    df['lastratingdate']=df['lastratingdate'].str.replace('-','/')
    df['lastratingdate']=pd.to_datetime(df['lastratingdate'],dayfirst=True)
    return df
def modif_drmk(df):
    df['txtval']=df['txt'].str.len()
    df['remarkDate']=df['remarkDate'].str.replace('-','/')
    df['remarkDate']= pd.to_datetime(df['remarkDate'],dayfirst=True)
    return df
def modif_drmk_supp(df): #replace T/F (dtype=boolean) by 1 or 0
    df.replace(to_replace=True, value=1, inplace=True)
    df.replace(to_replace=False, value=0, inplace=True) 
    return df

In [8]:
dtest=basicClean(dtest)
dtrain=basicClean(dtrain)
drating=basicClean(drating)
drmk=basicClean(drmk)
drmk_supp=basicClean(drmk_supp)

In [9]:
dtrain=clean_dt(dtrain)
dtest=clean_dt(dtest)

In [10]:
drmk=modif_drmk(drmk)
drmk_supp=modif_drmk_supp(drmk_supp)

In [11]:
# print(len(drmk_supp['remarkId'].unique())) #37178
# print(len(drmk['remarkId'].unique())) #38993
#  37178 and 38993. :(, there are some remarkIds, for which ther's no support value :(

In [12]:
# print(len(dtrain)/(len(dtrain['emp'].unique()))) #4.938375350140056

In [13]:
# print(np.sum(dtrain.isnull()))
# print(np.sum(drating.isnull()))
# print(np.sum(drmk.isnull())) #it has nan values
# print(np.sum(drmk_supp.isnull()))

In [272]:
print(len(drating['comp']))
print(len(drmk['comp']))
print(len(drmk_supp['comp']))
print('')

print(len(drating['comp'].unique()))
print(len(drmk['comp'].unique()))
print(len(drmk_supp['comp'].unique()))
a=np.array(drating['comp'].unique())
b=np.array(drmk['comp'].unique())
c=np.array(drmk_supp['comp'].unique())
A=np.concatenate((c,np.concatenate((a,b))))
B=set(A)-set(a)
len(B)

221232
38993
324478

37
36
34


0

## features
###  drating

In [14]:
drating=drating.sort_values(['compemp','Date'], ascending=[True, False])
drating['const']=1
# drating

In [15]:
# 24Jul latest
# RtComp_Dict comp:[noOfEmp,noOfRt,avgRt,lastRt Date]
empCount_df=drating.drop_duplicates(subset='compemp',keep='first')
empCount_df=empCount_df.groupby('comp').sum().reset_index()
RtCount_df=drating.groupby('comp').sum().reset_index()
RtAvg_df=drating.groupby('comp').mean().reset_index()
RtDt_df=drating.drop_duplicates(subset='comp',keep='first')
RtComp_Dict={}

RtDt_df['Date']=RtDt_df['Date'].str.replace('-','/')
RtDt_df['Date']=pd.to_datetime(RtDt_df['Date'],dayfirst=True)
fromDate=min(RtDt_df['Date'])
RtDt_df['Date']=(RtDt_df['Date']-fromDate).dt.days.astype(int)


for i in range(len(RtDt_df)):
    ar=[empCount_df.iloc[i]['const'],RtCount_df.iloc[i]['const'],round(RtAvg_df.iloc[i]['rating'],2),RtDt_df.iloc[i]['Date']]
    RtComp_Dict[RtDt_df.iloc[i]['comp']]=ar
# RtComp_Dict

/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [16]:
# 24jul latest
# RtEmp_Dict compemp: [noOfRatings, avgRating, recRating, lastDate]
RtEmp_df=drating.groupby('compemp').sum().reset_index()
# RtEmp_df
RtEmpRec_df=drating.drop_duplicates(subset='compemp',keep='first')
# RtEmpRec_df


RtEmpRec_df['Date']=RtEmpRec_df['Date'].str.replace('-','/')
RtEmpRec_df['Date']=pd.to_datetime(RtEmpRec_df['Date'],dayfirst=True)
fromDate=min(RtEmpRec_df['Date'])
RtEmpRec_df['Date']=(RtEmpRec_df['Date']-fromDate).dt.days.astype(int)


RtEmp_Dict={} #compemp: [noOfRatings, avgRating,recRating, lastDate]
for i in range(len(RtEmp_df)):
    ar=[RtEmp_df.iloc[i]['const'],round((RtEmp_df.iloc[i]['rating']/RtEmp_df.iloc[i]['const']),2),RtEmpRec_df.iloc[i]['rating'],RtEmpRec_df.iloc[i]['Date']]
    RtEmp_Dict[RtEmp_df.iloc[i]['compemp']]=ar
# RtEmp_Dict

/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/rudransh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

### dremark

In [17]:
drmk=drmk[['comp','compemp','txtval','remarkId','remarkDate']]
# print(np.sum(drmk.isnull()))
# print('')
# print(drmk.describe())
# print("______")
# print(len(drmk))
drmk.dropna(axis=0,thresh=3,inplace=True)
# print(np.sum(drmk.isnull()))
# print("______")
# print(len(drmk))
# drmk

## drmk_supp

In [18]:
# drmk_supp
# support + oppose =1

In [19]:
rmk_supp_df=drmk_supp.groupby('remarkId').sum().reset_index()
rmk_supp_df
rmk_supp_Dict={} #remarkId:[support,oppose]
for i in range(len(rmk_supp_df)):
    ar=[rmk_supp_df.iloc[i]['support'],rmk_supp_df.iloc[i]['oppose']]
    rmk_supp_Dict[rmk_supp_df.iloc[i]['remarkId']]=ar
# rmk_supp_Dict

In [20]:
emp_supp_df=drmk_supp.groupby('compemp').sum().reset_index()
# emp_supp_df
emp_supp_Dict={} #compemp:[support, oppose]
for i in range(len(emp_supp_df)):
    ar=[emp_supp_df.iloc[i]['support'],emp_supp_df.iloc[i]['oppose']]
    emp_supp_Dict[emp_supp_df.iloc[i]['compemp']]=ar
# emp_supp_Dict
# emp_supp_Dict compemp:[support, oppose]

In [21]:
# drmk_supp
compR=[]
compempR=[]
compempRDt=[]
for i in range(len(drmk_supp)):
    try:
        compR.append(RtComp_Dict[drmk_supp.iloc[i]['comp']][2])
    except KeyError:
        compR.append(np.nan)
    
for i in range(len(drmk_supp)):
    try:
        compempR.append(RtEmp_Dict[drmk_supp.iloc[i]['compemp']][1])
        compempRDt.append(RtEmp_Dict[drmk_supp.iloc[i]['compemp']][3])
    except KeyError:
        compempR.append(np.nan)
        compempRDt.append(np.nan)
# compempRDt

In [22]:
# drmk_supp

In [23]:
# THIS PART TAKES EXCEPTIONALLY LONG TIME TO PROCESS!
drmk_supp['compR']=compR
drmk_supp['compempR']=compempR
drmk_supp['compempRDt']=compempRDt
drmk_supp['compR*supp']=drmk_supp['compR']*drmk_supp['support']
drmk_supp['compR*opp']=drmk_supp['compR']*drmk_supp['oppose']
drmk_supp['compempR*supp']=drmk_supp['compempR']*drmk_supp['support']
drmk_supp['compempR*opp']=drmk_supp['compempR']*drmk_supp['oppose']
drmk_supp['compempDt*supp*compempR']=drmk_supp['compempR']*drmk_supp['support']
drmk_supp['compempDt*opp*compempR']=drmk_supp['compempR']*drmk_supp['oppose']
drmk_supp=drmk_supp.fillna(drmk_supp.mean())
# drmk_supp


In [24]:
# drmk_supp
rmk_aux_df=drmk_supp.groupby('remarkId').mean().reset_index()
# rmk_aux_df


In [25]:
rmk_aux_Dict={}
for i in range(len(rmk_aux_df)):
    rId=rmk_aux_df.iloc[i]['remarkId']
    ar=[round(rmk_aux_df.iloc[i]['compR'],2),round(rmk_aux_df.iloc[i]['compempR'],2),round(rmk_aux_df.iloc[i]['compempR*supp'],2),round(rmk_aux_df.iloc[i]['compempR*opp'],2),round(rmk_aux_df.iloc[i]['compempDt*supp*compempR'],2),round(rmk_aux_df.iloc[i]['compempDt*opp*compempR'],2)]
    rmk_aux_Dict[rId]=ar
# rmk_aux_Dict

## drmk and drmk_supp

In [26]:
drmk=drmk.sort_values(['compemp','remarkDate'], ascending=[True, False])
# drmk

In [27]:
drmk=drmk.drop_duplicates(subset='remarkId', keep='first').reset_index()
drmk
supp_=[]
opp_=[]
supp_mean=[]

aux_0=[]
aux_1=[]
aux_2=[]
aux_3=[]
aux_4=[]
aux_5=[]

for i in range(len(drmk)):
    rId=drmk.iloc[i]['remarkId']
    try:
        ar=rmk_supp_Dict[rId]
        supp_.append(ar[0])
        opp_.append(ar[1])
        supp_mean.append(ar[0]/(ar[0]+ar[1]))
        
        ar2=rmk_aux_Dict[rId]
        aux_0.append(ar2[0])
        aux_1.append(ar2[1])
        aux_2.append(ar2[2])
        aux_3.append(ar2[3])
        aux_4.append(ar2[4])
        aux_5.append(ar2[5])
        
        
        
    except KeyError:
        supp_.append(np.nan)
        opp_.append(np.nan)
        supp_mean.append(np.nan)
        
        aux_0.append(np.nan)
        aux_1.append(np.nan)
        aux_2.append(np.nan)
        aux_3.append(np.nan)
        aux_4.append(np.nan)
        aux_5.append(np.nan)
        
        
        
drmk['support']=supp_
drmk['oppose']=opp_
drmk['supp_mean']=supp_mean

drmk['aux_0']=aux_0
drmk['aux_1']=aux_1
drmk['aux_2']=aux_2
drmk['aux_3']=aux_3
drmk['aux_4']=aux_4
drmk['aux_5']=aux_5


drmk=drmk.fillna(drmk.mean())
# drmk

In [28]:
drmk=drmk.round({'support':0,'oppose':0,'supp_mean':3,'aux_0':3,'aux_1':3,'aux_2':3,'aux_3':3,'aux_4':3,'aux_5':3})
# drmk

In [29]:
drmk['txt*supp']=drmk['txtval']*drmk['supp_mean']
# drmk
RmkComp_sum_df=drmk.groupby('comp').sum().reset_index()
RmkComp_mean_df=drmk.groupby('comp').mean().reset_index()
RmkComp_Dict={} # comp:[supp, opp, supp_mean, avg txtval, avg(txtval*supp_mean)]
for i in range(len(RmkComp_sum_df)):
    supp_total=RmkComp_sum_df.iloc[i]['support']
    opp_total=RmkComp_sum_df.iloc[i]['oppose']
    supp_mean_=round((supp_total/(supp_total+opp_total)),3)
    avgtxtval=(RmkComp_mean_df.iloc[i]['txtval'])
    
    c_aux0=round(RmkComp_mean_df.iloc[i]['aux_0'],3)
    c_aux1=round(RmkComp_mean_df.iloc[i]['aux_1'],3)
    c_aux2=round(RmkComp_mean_df.iloc[i]['aux_2'],3)
    c_aux3=round(RmkComp_mean_df.iloc[i]['aux_3'],3)
    c_aux4=round(RmkComp_mean_df.iloc[i]['aux_4'],3)
    c_aux5=round(RmkComp_mean_df.iloc[i]['aux_5'],3)
    
    ar=[supp_total,opp_total,supp_mean_,round(avgtxtval,0),round(avgtxtval*supp_mean_,2),c_aux0,c_aux1,c_aux2,c_aux3,c_aux4,c_aux5]
    
    RmkComp_Dict[RmkComp_sum_df.iloc[i]['comp']]=ar
# RmkComp_Dict
# RmkComp_Dict comp:[supp, opp, supp_mean, avg txtval, avg(txtval*supp_mean), more]

In [30]:
RmkEmp_rec_Dict={} # RmkEmp_rec_Dict Compemp:[txtval, supp, opp, supp_mean, txt*supp, lastdate]
RmkEmp_avg_Dict={} # RmkEmp_avg_Dict Compemp:[avgtxtval, supp, opp, supp_mean, avg(txt*supp), more]
RmkEmp_rec_df=drmk.drop_duplicates(subset='compemp', keep='first').reset_index()
RmkEmp_avg_df=drmk.groupby('compemp').mean().reset_index()
# RmkEmp_avg_df


# RmkEmp_rec_df['remarkDate']=RmkEmp_rec_df['remarkDate'].str.replace('-','/')
RmkEmp_rec_df['remarkDate']=pd.to_datetime(RmkEmp_rec_df['remarkDate'],dayfirst=True)
fromDate=min(RmkEmp_rec_df['remarkDate'])
RmkEmp_rec_df['remarkDate']=(RmkEmp_rec_df['remarkDate']-fromDate).dt.days.astype(int)

for i in range(len(RmkEmp_rec_df)):
    ar1=[RmkEmp_rec_df.iloc[i]['txtval'],RmkEmp_rec_df.iloc[i]['support'],RmkEmp_rec_df.iloc[i]['oppose'],RmkEmp_rec_df.iloc[i]['supp_mean'],RmkEmp_rec_df.iloc[i]['txt*supp'],RmkEmp_rec_df.iloc[i]['remarkDate']]
    e_aux0=round(RmkEmp_avg_df.iloc[i]['aux_0'],3)
    e_aux1=round(RmkEmp_avg_df.iloc[i]['aux_1'],3)
    e_aux2=round(RmkEmp_avg_df.iloc[i]['aux_2'],3)
    e_aux3=round(RmkEmp_avg_df.iloc[i]['aux_3'],3)
    e_aux4=round(RmkEmp_avg_df.iloc[i]['aux_4'],3)
    e_aux5=round(RmkEmp_avg_df.iloc[i]['aux_5'],3)
    
    ar2=[round(RmkEmp_avg_df.iloc[i]['txtval'],0),round(RmkEmp_avg_df.iloc[i]['support'],2),round(RmkEmp_avg_df.iloc[i]['oppose'],2),round(RmkEmp_avg_df.iloc[i]['supp_mean'],3),round(RmkEmp_avg_df.iloc[i]['txt*supp'],3),e_aux0,e_aux1,e_aux2,e_aux3,e_aux4,e_aux5]
    empId=RmkEmp_rec_df.iloc[i]['compemp']
    RmkEmp_rec_Dict[empId]=ar1
    RmkEmp_avg_Dict[empId]=ar2
# RmkEmp_rec_Dict

In [31]:
# RmkEmp_avg_Dict

In [32]:
# emp_supp_Dict compemp:[support, oppose]
# RmkEmp_rec_Dict Compemp:[txtval, supp, opp, supp_mean, txt*supp, lastdate]
# RmkEmp_avg_Dict Compemp:[avgtxtval, supp, opp, supp_mean, avg(txt*supp)]

In [33]:
# 5 aug : i have to append test and train data to extract a dictionary of last rating data

d1=dtrain.copy(deep=True)
d2=dtest.copy(deep=True)
d1=d1[['comp','compemp','lastratingdate']]
d2=d2[['comp','compemp','lastratingdate']]

frames = [d1,d2]
d_final = pd.concat(frames)

d_final['Dt_num']=pd.to_datetime(d_final['lastratingdate'],dayfirst=True)
fromDate=min(d_final['Dt_num'])
d_final['Dt_num']=(d_final['Dt_num']-fromDate).dt.days.astype(int)
d_final=d_final.sort_values(['comp','compemp'], ascending=[True, True])
# d_final



lRDt_emp_Dict={}
aRDt_comp_Dict={}
lRDt_emp_df=d_final.drop_duplicates(subset='compemp', keep='first').reset_index()
aRDt_comp_df=d_final.groupby('comp').mean().reset_index()
for i in range(len(lRDt_emp_df)):
    lRDt_emp_Dict[lRDt_emp_df.iloc[i]['compemp']]=lRDt_emp_df.iloc[i]['Dt_num']
for i in range(len(aRDt_comp_df)):
    aRDt_comp_Dict[aRDt_comp_df.iloc[i]['comp']]=aRDt_comp_df.iloc[i]['Dt_num']
# lRDt_emp_Dict

In [34]:
# len(lRDt_emp_Dict)  # 3505 initially
# len(lRDt_emp_Dict) # now it is 4374 :) 

In [35]:
# left ratio of company
# dtrain

# gives dict comp:[total left, left ratio]
def feat_leftRatio(dtrain):
    dtrain_left_sum=dtrain.groupby('comp').sum().reset_index()
    dtrain_left_mean=dtrain.groupby('comp').mean().reset_index()
    comp_left_Dict={}
    for i in range(len(dtrain_left_sum)):
        leftTotal=dtrain_left_sum.iloc[i]['left']
        leftMean=dtrain_left_mean.iloc[i]['left']
        comp_left_Dict[dtrain_left_sum.iloc[i]['comp']]=[leftTotal,round(leftMean,3)]
    return comp_left_Dict
comp_left_Dict=feat_leftRatio(dtrain)
# comp_left_Dict

## Extract features

In [36]:
def get_label(df):
    Y=[]    
    for i in range(len(df)):
        Y.append(df.iloc[i]['left'])
    return np.array(Y)

    
def extract_comp_feat(df,RtComp_Dict,RmkComp_Dict,comp_left_Dict,aRDt_comp_Dict):
# RtComp_Dict comp:[noOfEmp,noOfRt,avgRt,lastRt Date]
# RmkComp_Dict comp:[supp, opp, supp_mean, avg txtval, avg(txtval*supp_mean)]
# comp_left_Dict comp:[total left, left ratio]   
# RmkComp_Dict and RmkEmp_avg_Dict has got 6 more features. aux0 to aux5
    
    empCt=[]
    noRt=[]
    avgRt=[]
    lastRtDt=[]
    
    supp=[]
    opp=[]
    supp_m=[]
    txtval=[]
    txtval_wt=[]
    
    left_tot=[]
    left_rat=[]
    
    aRDt=[]
    
    c_aux_0=[]
    c_aux_1=[]
    c_aux_2=[]
    c_aux_3=[]
    c_aux_4=[]
    c_aux_5=[]
    
    
    for i in range(len(df)):
        compId=df.iloc[i]['comp']
        try:
            aRDt.append(aRDt_comp_Dict[compId])
        except KeyError:
            aRDt.append(np.nan)
        
    
    for i in range(len(df)):
        compId=df.iloc[i]['comp']
        try:
            ar=RtComp_Dict[compId]
            empCt.append(ar[0])
            noRt.append(ar[1])
            avgRt.append(ar[2])
            lastRtDt.append(ar[3])  
        except KeyError:
            empCt.append(np.nan)
            noRt.append(np.nan)
            avgRt.append(np.nan)
            lastRtDt.append(np.nan)  
            
    for i in range(len(df)):
        compId=df.iloc[i]['comp']
        try:
            ar=RmkComp_Dict[compId]
            supp.append(ar[0])
            opp.append(ar[1])
            supp_m.append(ar[2])
            txtval.append(ar[3])
            txtval_wt.append(ar[4])
            c_aux_0.append(ar[5])
            c_aux_1.append(ar[6])
            c_aux_2.append(ar[7])
            c_aux_3.append(ar[8])
            c_aux_4.append(ar[9])
            c_aux_5.append(ar[10])
 
        except KeyError:
            supp.append(np.nan)
            opp.append(np.nan)
            supp_m.append(np.nan)
            txtval.append(np.nan)
            txtval_wt.append(np.nan)
            c_aux_0.append(np.nan)
            c_aux_1.append(np.nan)
            c_aux_2.append(np.nan)
            c_aux_3.append(np.nan)
            c_aux_4.append(np.nan)
            c_aux_5.append(np.nan)
            

    for i in range(len(df)):
        compId=df.iloc[i]['comp']
        try:
            ar=comp_left_Dict[compId]
            left_tot.append(ar[0])
            left_rat.append(ar[1])
        except KeyError:
            left_tot.append(np.nan)
            left_rat.append(np.nan)
     
    return [empCt,
            noRt,
            avgRt,
            lastRtDt,
            supp,
            opp,
            supp_m,
            txtval,
            txtval_wt,
            left_tot,
            left_rat,
            aRDt,
            c_aux_0,
            c_aux_1,
            c_aux_2,
            c_aux_3,
            c_aux_4,
            c_aux_5]

In [37]:
def extract_emp_feat(df,RtEmp_Dict,emp_supp_Dict,RmkEmp_rec_Dict,RmkEmp_avg_Dict,lRDt_emp_Dict):
# RtEmp_Dict compemp: [noOfRatings, avgRating, recRating, lastDate]
# emp_supp_Dict compemp:[support, oppose]
# RmkEmp_rec_Dict Compemp:[txtval, supp, opp, supp_mean, txt*supp, lastdate]
# RmkEmp_avg_Dict Compemp:[avgtxtval, supp, opp, supp_mean, avg(txt*supp)]
# RmkComp_Dict and RmkEmp_avg_Dict has got 6 more features. aux0 to aux5   
    
    noOfRt=[]
    avgRt=[]
    recRt=[]
    RtlastDt=[]
    
    emp_supp=[]
    emp_opp=[]
    
    rec_txtval=[]
    rec_supp=[]
    rec_opp=[]
    rec_supp_mean=[]
    rec_txt_wt=[]
    RklastDt=[]
    
    avg_txtval=[]
    avg_supp=[]
    avg_opp=[]
    avg_supp_mean=[]
    avg_txt_wt=[]
    
    lRDt=[]
    
    e_aux_0=[]
    e_aux_1=[]
    e_aux_2=[]
    e_aux_3=[]
    e_aux_4=[]
    e_aux_5=[]
    
    
    for i in range(len(df)):
        empId=df.iloc[i]['compemp']
        try:
            lRDt.append(lRDt_emp_Dict[empId])
        except KeyError:
            lRDt.append(np.nan)
    
    
    
    for i in range(len(df)):
        empId=df.iloc[i]['compemp']
        try:
            ar=RtEmp_Dict[empId]
            noOfRt.append(ar[0])
            avgRt.append(ar[1])
            recRt.append(ar[2])
            RtlastDt.append(ar[3])
        except KeyError:
            noOfRt.append(np.nan)
            avgRt.append(np.nan)
            recRt.append(np.nan)
            RtlastDt.append(np.nan)
            
    for i in range(len(df)):
        empId=df.iloc[i]['compemp']
        try:
            ar=emp_supp_Dict[empId]
            emp_supp.append(ar[0])
            emp_opp.append(ar[1])
        except KeyError:
            emp_supp.append(np.nan)
            emp_opp.append(np.nan)
    
    for i in range(len(df)):
        empId=df.iloc[i]['compemp']
        try:
            ar=RmkEmp_rec_Dict[empId]
            rec_txtval.append(ar[0])
            rec_supp.append(ar[1])
            rec_opp.append(ar[2])
            rec_supp_mean.append(ar[3])
            rec_txt_wt.append(ar[4])
            RklastDt.append(ar[5])
        except KeyError:
            rec_txtval.append(np.nan)
            rec_supp.append(np.nan)
            rec_opp.append(np.nan)
            rec_supp_mean.append(np.nan)
            rec_txt_wt.append(np.nan)
            RklastDt.append(np.nan)
    
    for i in range(len(df)):
        empId=df.iloc[i]['compemp']
        try:
            ar=RmkEmp_avg_Dict[empId]
            avg_txtval.append(ar[0])
            avg_supp.append(ar[1])
            avg_opp.append(ar[2])
            avg_supp_mean.append(ar[3])
            avg_txt_wt.append(ar[4])
            e_aux_0.append(ar[5])
            e_aux_1.append(ar[6])
            e_aux_2.append(ar[7])
            e_aux_3.append(ar[8])
            e_aux_4.append(ar[9])
            e_aux_5.append(ar[10])
            
        except KeyError:
            avg_txtval.append(np.nan)
            avg_supp.append(np.nan)
            avg_opp.append(np.nan)
            avg_supp_mean.append(np.nan)
            avg_txt_wt.append(np.nan)
            e_aux_0.append(np.nan)
            e_aux_1.append(np.nan)
            e_aux_2.append(np.nan)
            e_aux_3.append(np.nan)
            e_aux_4.append(np.nan)
            e_aux_5.append(np.nan)
    
    return [noOfRt,
            avgRt,
            recRt,
            RtlastDt,
            emp_supp,
            emp_opp,
            rec_txtval,
            rec_supp,
            rec_opp,
            rec_supp_mean,
            rec_txt_wt,
            RklastDt,
            avg_txtval,
            avg_supp,
            avg_opp,
            avg_supp_mean,
            avg_txt_wt,
            lRDt,
            e_aux_0,
            e_aux_1,
            e_aux_2,
            e_aux_3,
            e_aux_4,
            e_aux_5]
    
  

In [38]:
train_label=get_label(dtrain)

In [170]:
def basicFeat(df):
    compFeat=extract_comp_feat(df,RtComp_Dict,RmkComp_Dict,comp_left_Dict,aRDt_comp_Dict)
    empFeat=extract_emp_feat(df,RtEmp_Dict,emp_supp_Dict,RmkEmp_rec_Dict,RmkEmp_avg_Dict,lRDt_emp_Dict)
    feat_df=pd.DataFrame({'empCt':compFeat[0],
                          'noRt':compFeat[1],
                          'avgRt':compFeat[2],
                          'lastRtDt':compFeat[3],
                          'supp':compFeat[4],
                          'opp':compFeat[5],
                          'supp_m':compFeat[6],
                          'txtval':compFeat[7],
                          'txtval_wt':compFeat[8],
                          'left_tot':compFeat[9],
                          'left_rat':compFeat[10],
                          'ARtDt':compFeat[11],
                            'c_aux_0':compFeat[12],
                            'c_aux_1':compFeat[13],
                            'c_aux_2':compFeat[14],
                            'c_aux_3':compFeat[15],
                            'c_aux_4':compFeat[16],
                            'c_aux_5':compFeat[17],
                          'noOfRt':empFeat[0],
                          'avgRt':empFeat[1],
                          'recRt':empFeat[2],
                          'RtlastDt':empFeat[3],
                          'emp_supp':empFeat[4],
                          'emp_opp':empFeat[5],
                          'rec_txtval':empFeat[6],
                          'rec_supp':empFeat[7],
                          'rec_opp':empFeat[8],
                          'rec_supp_mean':empFeat[9],
                          'rec_txt_wt':empFeat[10],
                          'RklastDt':empFeat[11],
                          'avg_txtval':empFeat[12],
                          'avg_supp':empFeat[13],
                          'avg_opp':empFeat[14],
                          'avg_supp_mean':empFeat[15],
                          'avg_txt_wt':empFeat[16],
                          'lRDt':empFeat[17],
#                          'e_aux_0':empFeat[18],
#                             'e_aux_1':empFeat[19],
#                             'e_aux_2':empFeat[20],
#                             'e_aux_3':empFeat[21],
#                             'e_aux_4':empFeat[22],
#                             'e_aux_5':empFeat[23]
                         })     
    
    feat_df.fillna(feat_df.mean(),inplace=True)
    feat_df['left_tot']=feat_df['left_tot']*feat_df['left_tot']
    feat_df['left_rat']=feat_df['left_rat']
    feat_df['emp_rating*txtval']=feat_df['avgRt']*feat_df['avg_txtval']
    feat_df['emp_rating*txtval_wt']=feat_df['avgRt']*feat_df['avg_txt_wt']
    feat_df['emp_raing*mean_supp']=feat_df['avgRt']*feat_df['avg_supp_mean']
    feat_df['comp_rating*txtval']=feat_df['avgRt']*feat_df['txtval']
    feat_df['comp_rating*txtval_wt']=feat_df['avgRt']*feat_df['txtval_wt']
    feat_df['comp_raing*mean_supp']=feat_df['avgRt']*feat_df['supp_m']
    
    feat_df['comp_raing*mean_supp']=feat_df['comp_raing*mean_supp']*100
    feat_df = feat_df.round({'lastRtDt': 0,'RtlastDt':0,'RklastDt':0,'avg_supp':2,'avg_opp':2,'avg_supp_mean':3,
                            'avg_txt_wt':0,'emp_rating*txtval':1,'emp_rating*txtval_wt':2,'emp_raing*mean_supp':2,
                            'comp_raing*mean_supp':3,})
    
    return feat_df


In [171]:
train_feat_df=basicFeat(dtrain)
test_feat_df=basicFeat(dtest)

In [172]:
# test_feat_df

In [173]:
# test_feat_df['lRDt'] i messed up here change the dictionary man all 797.307 values
# test_feat_df['lRDt'] #yup its good now

In [174]:
train_analy=basicFeat(dtrain)
train_analy['left']=train_label
# train_analy=train_analy.groupby('left').mean().reset_index()
# train_analy

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns


In [159]:
train_analy=train_analy.groupby('left').mean().reset_index()
# train_analy

# Final

In [212]:
def save_csv(Y_pred,filename):
    output=pd.DataFrame({'id':dtest['id'],'left':Y_pred})
    output.to_csv(filename,index=False)

In [47]:
def wt_acc(Y_true,Y_pred):
    w=[1,5] # weight for class 0 is 1 for class 1 is 5
    n=len(Y_pred)
    num=0
    den=0
    for i in range(len(Y_pred)):
        den = den + w[Y_true[i]]
        if Y_pred[i]==Y_true[i]:
            num = num + w[Y_pred[i]]
    return num/(den)


In [69]:
def conf(Y_val, Y_val_pred):
#     print(np.sum(Y_val==1))
#     print(np.sum(Y_val_pred==1))

    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(Y_val, Y_val_pred)
    import seaborn as sns
    import matplotlib.pyplot as plt     

    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax) #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix')



In [175]:
X_train_total=train_feat_df.to_numpy()
Y_train_total=get_label(dtrain)
X_test=test_feat_df.to_numpy()

In [176]:
# split the set to training and validation, set accuracy criteria
X_train, X_val, Y_train, Y_val = train_test_split(X_train_total, Y_train_total, test_size=0.33, random_state=5)

In [51]:
# print(test_feat_df.var())

In [222]:
len(X_train[0])

41

### Validation

In [52]:
scorer = sklearn.metrics.make_scorer(wt_acc)

In [287]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 5)
from pprint import pprint

print('Params in use:\n')
pprint(rf.get_params())
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]
min_impurity_decrease=[0.0001]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'class_weight':[{0:1,1:7}],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion':['entropy','gini'],
               'bootstrap': bootstrap,
              'min_impurity_decrease':min_impurity_decrease}
pprint(random_grid)

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=5, n_jobs = -1,scoring=scorer)

rf_random.fit(X_train, Y_train)

Params in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 5,
 'verbose': 0,
 'warm_start': False}
{'bootstrap': [True, False],
 'class_weight': [{0: 1, 1: 7}],
 'criterion': ['entropy', 'gini'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_impurity_decrease': [0.0001],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [288]:
pr=rf_random.best_params_
pr

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'min_impurity_decrease': 0.0001,
 'max_features': 'auto',
 'max_depth': 20,
 'criterion': 'entropy',
 'class_weight': {0: 1, 1: 7},
 'bootstrap': False}

### random forest

In [2]:
# above code for validation gives different answers at diff times,
# below is implementation of one of them

# standard one
rfc=RandomForestClassifier(n_estimators=1000,min_samples_split=10,criterion='entropy',
    min_samples_leaf=4,max_features='auto',max_depth=50,class_weight={0:1,1:200},
                           random_state=5,bootstrap=True, min_impurity_decrease=0.0001)

rfc.fit(X_train, Y_train)

Y_val_pred_rfc=rfc.predict(X_val)

print("weighted accuracy is: "+str(wt_acc(Y_val,Y_val_pred_rfc)))

print("length of Y_val", len(Y_val))

NameError: name 'RandomForestClassifier' is not defined

In [309]:
print(np.sum(Y_val))

200


In [318]:
rfc.fit(X_train_total,Y_train_total)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 1, 1: 200}, criterion='entropy',
                       max_depth=50, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0001,
                       min_impurity_split=None, min_samples_leaf=4,
                       min_samples_split=10, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=None, oob_score=False,
                       random_state=5, verbose=0, warm_start=False)

In [255]:
print(len(X_train_total))
print(592+2934)
print(len(X_val))

3526
3526
1164


In [57]:
# 28 and 78 and 0.889

In [319]:
Y_pred=rfc.predict(X_test)
save_csv(Y_pred,'lastandFinal.csv')